< [Introduction](Introduction.ipynb) | [Contents](Contents.ipynb) | [Prepare the initial section of the model](STREAM_Initial.ipynb) >

# 2. The PCRaster Dynamic Modelling Framework
In the previous tutorials you've worked with the PCRaster library for map algebra. PCRaster's real power, however, is the dynamic modelling framework. In this section you'll learn more about the dynamic modelling framework. In the next secions we're going to build the STREAM model to simulate runoff in the Upper Mara River Basin, Kenya.

The PCRaster Dynamic Modelling Framework can be used with the following template:
```Python
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
  def __init__(self, cloneMap):
    DynamicModel.__init__(self)
    setclone(cloneMap)

  def initial(self):
      # Here you write the code that has to be run only once.
  
  def dynamic(self):
      # Here you write the code that needs to be executed every time step.
  
myModel = RunoffModel("mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=100, firstTimestep=1)
dynModelFw.run()
```


When you make your own model you need to change the following in this template:
1. Define the clone map. In this case it's called `mask.map`. All raster maps need to have the same properties as the clone map (i.e. same number of rows and collumns, coordinate system, extent, pixels size). PCRaster checks this when the code is run.
2. Define the time steps. Here `lastTimeStep=100` and `firstTimestep=1` means that the model runs 100 time steps starting at time step 1.
3. Under `def initial(self)` you write code that needs to be run only once, i.e. for model initialisation.
4. Under `def dynamic(self)` you write code that needs to be executed every time step., i.e. the iterations.

Let's have a look at how this works by running the code below.


In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        pass
        
        
        
    
    def dynamic(self):
        pass
        
        

  
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Note that for each iteration in the `dynamic` section it prints a `.`.

If we want to use variables and constants, we can do that in different ways.
Have a look at the script below and run it.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        a = 0
        b = a + 1
        print(b)
    
    def dynamic(self):
        c = 1
        d = c + 1
        print(d)
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

These are called *local variables*. They can only be used in the function where they are defined, but not outside. Check what happens if we run the code below.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        a = 0
    
    def dynamic(self):
        b = a + 1
        print(b)
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

`NameError: name 'a' is not defined` means that in line 13 Python doesn't know variable `a` yet. If we want to use variable `a` in the `dynamic` section, we need to make it a *global variable*. We do this by adding `self.` before the variable. Check the modified script below and run it.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        self.a = 0
    
    def dynamic(self):
        b = self.a + 1
        print(b)
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Now we can easily increase variable `b` each time step. Check the following code and run it.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        self.a = 0
    
    def dynamic(self):
        b = self.a + 1
        self.a = b
        print(self.a)
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

In the map algebra tutorial you've learned that with `readmap` you can read PCRaster maps from disk and with `report` you can write PCRaster maps to disk. In the dynamic modelling framework this is still the case, but the format it reads and write depends on where you use `readmap` and `report`.

In the `initial` section we can read land-use map and soil map from disk using the code below. You can run it to see if it works.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        self.landuse = self.readmap("./Data/landuse")
        self.soil = self.readmap("./Data/soil")
    
    def dynamic(self):
        pass
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Although we didn't give the file extension, the script runs without errors. That is because the framework understands that if you use `self.readmap` in the `initial` section it will always be a static map with the `.map` file extension.

It will also understand that if you use `self.readmap` in the `dynamic` section that it needs to look for PCRaster dynamic map stacks. These time series of maps have a naming convention: at least one alphabetic character followed by zeros until the total number of characters before the dot is eight. After the dot (the file extention part of the file name) it will have 001 for the first time step, 002 for the second, etc.

In the `Data` folder you can find a PCRaster dynamic map stack for precipitation (mm/10 days). These are the following files for the 10 time steps of our model:
```
pr000000.001
pr000000.002
pr000000.003
pr000000.004
pr000000.005
pr000000.006
pr000000.007
pr000000.008
pr000000.009
pr000000.010
```

Let's read them in the `dynamic` section and see if the script runs.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr") 
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

The script runs without errors and both the static and dynamic maps have been read and we can use them in the script.

Let's use a lookup table to create a map with the interception threshold from the land-use map. We'll use the land-use map from [FAO WaPOR](https://wapor.apps.fao.org/). In the `Data` folder you can find `d.tbl` which has the interception threshold for different land-use types.

| Cell value | Land use | Interception threshold (mm)
| ---------- | -------- | ---------------------------
| 1 | Forest | 60
| 2 | Bushland dense | 50
| 3 | Woodland | 40
| 4 | Bushland sparse | 40
| 5 | Grassland | 10
| 11 | Dense agriculture | 25
| 12 | Sparse agriculture | 20
| 13 | Plantation | 15


We also write the result to disk using `self.report`.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)
        self.report(self.InterceptionThreshold,"./Data/d")
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr") 
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Visualise the land-use map and map with the interception threshold.

In [ ]:
aguila("./Data/landuse.map","./Data/d.map")

Now we're a bit more familiar with the PCRaster Dynamic Modelling Framework we can start further developing the `initial` section of the model.

< [Introduction](Introduction.ipynb) | [Contents](Contents.ipynb) | [Prepare the initial section of the model](STREAM_Initial.ipynb) >